In [ ]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,desc,sum,to_date,date_format,concat_ws,format_string,to_timestamp,year,when,lit,month,initcap,upper,lower
from pyspark.sql import Window
import pandas as pd

# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)


# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)

In [ ]:
# tmp = upper(lit('StrinG'))
# display(tmp)

Column<'upper(StrinG)'>

In [ ]:
## TABLA SALES
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/Completo_ventas/Ds_G_sales.parquet'
sales = spark.read.parquet(inpath)
sales.createOrReplaceTempView("sales_vw")
#print(sales.count())
#display(sales.limit(100))


## MAESTRA LOCALIDADES AGRUPADAS MOVISTAR
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Maestras/Localidades_Movistar/localidades_agrupadas_movistar.csv'
localidad_movistar = spark.read.csv(inpath, sep = '|', header=True, inferSchema=True)
print(localidad_movistar.count())
localidad_movistar.createOrReplaceTempView("localidad_movistar_vw")

1219


In [ ]:
# tmp2 = localidad_movistar.filter(col('LOGICAL_SEAT_CATEGORY').like('%CAGADAS%'))
# display(tmp2)
# #display(tmp)

In [ ]:
# # Agregar una columna que indique si es duplicado o no
# window_spec = Window().partitionBy("LOGICAL_SEAT_CATEGORY").orderBy("LOGICAL_SEAT_CATEGORY")

# df_spark = localidad_movistar.withColumn("es_duplicado", count("LOGICAL_SEAT_CATEGORY").over(window_spec) > 1)

# # Filtrar solo las filas duplicadas
# duplicados = df_spark.filter("es_duplicado")#.select('Evento').distinct()

# display(duplicados)
# #display(temp.limit(100))

In [ ]:
# tmp = sales.filter(col('T_PRODUCT_ID').like('%10229437245287%'))
#     #select('PRODUCT','TOWN','TOWN_N').\

# display(tmp.limit(100))
# #display(tmp)

# tmp1 = tmp.select('Key_aforo').distinct()
# display(tmp1)

In [ ]:
eventos = spark.sql('''

WITH tmp11 as (
    SELECT
    Key_evento,
    Key_aforo,
    LOGICAL_SEAT_CATEGORY,
    Nuevo_aforo as Aforo,    
    SUM(NET_SOLD_C_QTY) Cortesias,
    --SUM(CHARGE_AMT_ITX) Booking_fee,
    SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados
    FROM sales_vw s
    GROUP BY Key_evento,Key_aforo,LOGICAL_SEAT_CATEGORY,Nuevo_aforo),
 
 tmp12 as (
	SELECT 
    Key_evento,
    SUM(Aforo) as Aforo,
    --SUM(Cortesias) as Cortesias,
    --SUM(Tickets_comprados) as Tickets_comprados,
    SUM(Tickets_comprados) / SUM(Aforo) as perc_ocupacion_comprados,
    (SUM(Tickets_comprados) + SUM(Cortesias)) / SUM(Aforo) as perc_ocupacion_emitidos
	FROM tmp11
	group by Key_evento),
 
 tmp13 as (
    SELECT
    t1.*,
    t2.Aforo as Aforo_total,
    t1.Tickets_comprados / t1.Aforo as perc_ocupacion_comprados_cl,
    (t1.Tickets_comprados + t1.Cortesias) / t1.Aforo as perc_ocupacion_emitidos_cl,
    t2.perc_ocupacion_comprados,
    t2.perc_ocupacion_emitidos
    FROM tmp11 t1
    LEFT JOIN tmp12 t2
      ON (t1.Key_evento = t2.Key_evento)),
 
 eventos as (
    SELECT
    s.Key_evento,
    s.Key_aforo,
    s.LOGICAL_SEAT_CATEGORY,--
    s.T_PRODUCT_ID AS CODIGO_DEL_EVENTO,
    UPPER(s.PRODUCT) AS EVENTO,
    s.T_PERFORMANCE_ID as CODIGO_SESION,
    s.ORGANIZATION AS ORGANISMO,
    s.SEASON as Temporada,
    s.PRODUCT_FAMILY AS FAMILIA_DEL_PRODUCTO,
    s.PRODUCT_CODE AS NICKNAME,
    s.LICENCE_NUMBER AS CODIGO_PULEP,
    '' as SEGMENTO_old,
    s.TOPIC_PPTO as SEGMENTO,
    s.SUB_TOPIC as SUBSEGMENTO,
    --TO_CHAR(TO_DATE(REPLACE(s.PRODUCT_DATE,'.','/'),'DD/MM/RR'), 'DD/MM/YY') AS FECHA_DEL_EVENTO,
    s.Fecha_evento AS FECHA_DEL_EVENTO,
    s.PRODUCT_TIME as Hora_evento,
    s.MATCH_DAY AS DIA_DE_JUEGO,
    s.TEAM_1 AS EQUIPO_1,
    s.TEAM_2_N AS EQUIPO_2,
    s.SITE AS VENUE,
    s.TOWN_N as Ciudad,
    --s.PRODUCT_STATE as ESTADO,
    '' as ESTADO,
    --s.PERFORMANCE_QUOTA as Aforo_disponible,
    s.INVOICE_CONTACT_FMT_NAME as Promotor,
    SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados,
    SUM(s.NET_SOLD_C_QTY) AS Cortesias,
    SUM(s.NET_SOLD_T_QTY) AS Total_emitido,
    SUM(s.NET_SOLD_TKT_AMT_ITX) AS Recaudo,
    MIN(s.REFERENCE_DATE) As FECHA_VALIDACION,
    MAX(s.REFERENCE_DATE) As ULTIMA_FECHA_VALIDACION
    FROM sales_vw s
    WHERE s.T_OPERATION_TYPE IN ('SALE','REFUND')
        AND s.T_OPERATION_KIND IN ('SINGLE_ENTRY','PRODUCT_COMPOSITION','SIMPLE_PRODUCT')--'COMPOSED_PRODUCT' 
        AND s.PRODUCT_CODE != 'CAN_INS'
        AND s.product_family IN ('Competición','Evento','Pase de visita')
		AND UPPER(s.PRODUCT) NOT LIKE '%PRUEBA%'
        AND UPPER(s.PRODUCT) NOT LIKE '%SUITE%'
        AND UPPER(s.PRODUCT) NOT LIKE '%PARQUEADERO%'
        AND UPPER(s.PRODUCT) NOT LIKE '%PREIMPRESO%'
        AND UPPER(s.PRODUCT) NOT LIKE '%ACREDITAC%'
        AND UPPER(s.PRODUCT) NOT LIKE '%REDENCI%'
        AND UPPER(s.PRODUCT) NOT LIKE '%ZONAS VIP%'
        AND UPPER(s.PRODUCT) NOT LIKE '%PCR%'
        AND UPPER(s.PRODUCT) NOT LIKE '%GENOS%'
        AND UPPER(s.PRODUCT) NOT LIKE '%TUBOLETA TE LLEVA%'
        AND UPPER(s.PRODUCT) NOT LIKE '%CONCIERGE%'
        AND UPPER(s.PRODUCT) NOT LIKE '%VMOVE%'
    GROUP BY
    s.Key_evento,s.Key_aforo,s.LOGICAL_SEAT_CATEGORY,s.T_PRODUCT_ID,s.PRODUCT,s.T_PERFORMANCE_ID,s.ORGANIZATION,s.SEASON,s.PRODUCT_FAMILY,
    s.PRODUCT_CODE,s.LICENCE_NUMBER,s.TOPIC_PPTO,s.SUB_TOPIC,s.Fecha_evento,s.PRODUCT_TIME,s.MATCH_DAY,s.TEAM_1,
    s.TEAM_2_N,s.TOWN_N,s.SITE,s.INVOICE_CONTACT_FMT_NAME
 )
 
SELECT
e.*,
t.Aforo,
t.Aforo_total,
t.perc_ocupacion_comprados_cl,
t.perc_ocupacion_emitidos_cl,
t.perc_ocupacion_comprados,
t.perc_ocupacion_emitidos
FROM eventos e
LEFT JOIN tmp13 t
  ON (e.Key_aforo == t.Key_aforo)
ORDER BY perc_ocupacion_comprados desc

''')


from pyspark.sql.functions import expr

eventos = eventos.withColumn(
    "Fecha Actualizacion",
    expr("current_timestamp() - interval 5 hours")
)

eventos.createOrReplaceTempView("eventos_vw")
print(eventos.count())
#display(eventos.limit(100))

148603


In [ ]:
tmp = eventos.filter((col('Key_evento').like('%10229119820162%'))) #&
                     #(col('Aforo').like('%1392%')))
# tmp = eventos.select('Key_evento','perc_ocupacion_emitidos').\
#     orderBy(desc('perc_ocupacion_emitidos')) #filter(col('perc_ocupacion_comprados') > 1)
display(tmp)

Key_evento,Key_aforo,LOGICAL_SEAT_CATEGORY,CODIGO_DEL_EVENTO,EVENTO,CODIGO_SESION,ORGANISMO,Temporada,FAMILIA_DEL_PRODUCTO,NICKNAME,CODIGO_PULEP,SEGMENTO_old,SEGMENTO,SUBSEGMENTO,FECHA_DEL_EVENTO,Hora_evento,DIA_DE_JUEGO,EQUIPO_1,EQUIPO_2,VENUE,Ciudad,ESTADO,Promotor,Tickets_comprados,Cortesias,Total_emitido,Recaudo,FECHA_VALIDACION,ULTIMA_FECHA_VALIDACION,Aforo,Aforo_total,perc_ocupacion_comprados_cl,perc_ocupacion_emitidos_cl,perc_ocupacion_comprados,perc_ocupacion_emitidos,Fecha Actualizacion
10229119820162|10229119824808,10229119820162|10229119824808|PLATEA 104 - 106,PLATEA 104 - 106,10229119820162,PRESIDANTE TOUR 2023,10229119824808,Tuboleta.com,2023,Evento,BMADANTE,null,,"Foros, Seminarios y Congresos",FORO,2023-08-30T20:00:00Z,20:00,null,null,null,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,,ALIVE PRODUCTIONS LOGISTICS & BOOKING SAS ( 5335361 ),748.0,0.0,748.0,2.2061875E8,2023-04-15T00:00:00Z,2023-06-20T00:00:00Z,748.0,6435.0,1.0,1.0,1.4197358197358196,1.5020979020979022,2024-06-04T09:05:29.925Z
10229119820162|10229119824808,10229119820162|10229119824808|FLOOR 2 SECTION 206 -214,FLOOR 2 SECTION 206 -214,10229119820162,PRESIDANTE TOUR 2023,10229119824808,Tuboleta.com,2023,Evento,BMADANTE,null,,"Foros, Seminarios y Congresos",FORO,2023-08-30T20:00:00Z,20:00,null,null,null,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,,ALIVE PRODUCTIONS LOGISTICS & BOOKING SAS ( 5335361 ),1630.0,7.0,1637.0,2.2498375E8,2023-04-14T00:00:00Z,2023-08-30T00:00:00Z,1639.0,6435.0,0.9945088468578401,0.9987797437461867,1.4197358197358196,1.5020979020979022,2024-06-04T09:05:29.925Z
10229119820162|10229119824808,10229119820162|10229119824808|PRESIDANTE 101 -103,PRESIDANTE 101 -103,10229119820162,PRESIDANTE TOUR 2023,10229119824808,Tuboleta.com,2023,Evento,BMADANTE,null,,"Foros, Seminarios y Congresos",FORO,2023-08-30T20:00:00Z,20:00,null,null,null,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,,ALIVE PRODUCTIONS LOGISTICS & BOOKING SAS ( 5335361 ),1139.0,322.0,1461.0,4.7025E8,2023-04-14T00:00:00Z,2023-08-30T00:00:00Z,1532.0,6435.0,0.7434725848563969,0.9536553524804178,1.4197358197358196,1.5020979020979022,2024-06-04T09:05:29.925Z
10229119820162|10229119824808,10229119820162|10229119824808|TRIBUNA FAN SUR,TRIBUNA FAN SUR,10229119820162,PRESIDANTE TOUR 2023,10229119824808,Tuboleta.com,2023,Evento,BMADANTE,null,,"Foros, Seminarios y Congresos",FORO,2023-08-30T20:00:00Z,20:00,null,null,null,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,,ALIVE PRODUCTIONS LOGISTICS & BOOKING SAS ( 5335361 ),53.0,102.0,155.0,1.855E7,2023-04-16T00:00:00Z,2023-08-30T00:00:00Z,155.0,6435.0,0.3419354838709677,1.0,1.4197358197358196,1.5020979020979022,2024-06-04T09:05:29.925Z
10229119820162|10229119824808,10229119820162|10229119824808|FLOOR 2 SECTION 201-205 & 215-219,FLOOR 2 SECTION 201-205 & 215-219,10229119820162,PRESIDANTE TOUR 2023,10229119824808,Tuboleta.com,2023,Evento,BMADANTE,null,,"Foros, Seminarios y Congresos",FORO,2023-08-30T20:00:00Z,20:00,null,null,null,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,,ALIVE PRODUCTIONS LOGISTICS & BOOKING SAS ( 5335361 ),2361.0,0.0,2361.0,2.0083375E8,2023-04-14T00:00:00Z,2023-04-28T00:00:00Z,2361.0,6435.0,1.0,1.0,1.4197358197358196,1.5020979020979022,2024-06-04T09:05:29.925Z
10229119820162|10229119824808,10229119820162|10229119824808|FLOOR 3 SECTION 308 - 312,FLOOR 3 SECTION 308 - 312,10229119820162,PRESIDANTE TOUR 2023,10229119824808,Tuboleta.com,2023,Evento,BMADANTE,null,,"Foros, Seminarios y Congresos",FORO,2023-08-30T20:00:00Z,20:00,null,null,null,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,,ALIVE PRODUCTIONS LOGISTICS & BOOKING SAS ( 5335361 ),3205.0,99.0,3304.0,3.4822325E8,2023-07-06T00:00:00Z,2023-08-30T00:00:00Z,null,6435.0,null,null,1.4197358197358196,1.5020979020979022,2024-06-04T09:05:29.925Z


In [ ]:
# tmp = spark.sql('''

#     SELECT
#     Key_evento,
#     Key_aforo,
#     PRODUCT,
#     PRODUCT_FAMILY,
#     Fecha_evento,
#     LOGICAL_SEAT_CATEGORY,
#     Nuevo_aforo as Aforo,    
#     SUM(NET_SOLD_C_QTY) Cortesias,
#     --SUM(CHARGE_AMT_ITX) Booking_fee,
#     SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados--,
#     --SUM(Tickets_comprados) / Aforo as perc_ocupacion_comprados,
#     --(SUM(Tickets_comprados) + SUM(Cortesias)) / Aforo as perc_ocupacion_emitidos
#     FROM sales_vw s
#     where Key_evento like '%10229406401500%' -- 10229196910853  10229362722009
#     GROUP BY Key_evento,Key_aforo,PRODUCT,PRODUCT_FAMILY,Fecha_evento,LOGICAL_SEAT_CATEGORY,Nuevo_aforo --10229385603077 ya quedo
# ''')

# display(tmp.limit(100))

In [ ]:
# tmp = spark.sql('''

# WITH tmp11 as (
#     SELECT
#     Key_evento,
#     Key_aforo,
#     LOGICAL_SEAT_CATEGORY,
#     Nuevo_aforo as Aforo,    
#     SUM(NET_SOLD_C_QTY) Cortesias,
#     --SUM(CHARGE_AMT_ITX) Booking_fee,
#     SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados
#     FROM sales_vw s
#     where Key_evento like '10229406401500%'
#     GROUP BY Key_evento,Key_aforo,LOGICAL_SEAT_CATEGORY,Nuevo_aforo)--,
 
#  --tmp12 as (
# 	SELECT 
#     Key_evento,
#     SUM(Aforo) as Aforo,
#     SUM(Cortesias) as Cortesias,
#     SUM(Tickets_comprados) as Tickets_comprados,
#     SUM(Tickets_comprados) / SUM(Aforo) as perc_ocupacion_comprados,
#     (SUM(Tickets_comprados) + SUM(Cortesias)) / SUM(Aforo) as perc_ocupacion_emitidos
# 	FROM tmp11
# 	group by Key_evento--),

# ''')

# print(tmp.count())
# display(tmp.limit(100))

In [ ]:
# tmp = spark.sql('''

# WITH tmp11 as (
#     SELECT
#     Key_evento,
#     Key_aforo,
#     LOGICAL_SEAT_CATEGORY,
#     Nuevo_aforo as Aforo,    
#     SUM(NET_SOLD_C_QTY) Cortesias,
#     --SUM(CHARGE_AMT_ITX) Booking_fee,
#     SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados
#     FROM sales_vw s
#     WHERE Key_evento like '%10228867039304%'
#     GROUP BY Key_evento,Key_aforo,LOGICAL_SEAT_CATEGORY,Nuevo_aforo),
 
#  tmp12 as (
# 	SELECT 
#     Key_evento,
#     --SUM(Aforo) as Aforo,
#     --SUM(Cortesias) as Cortesias,
#     --SUM(Tickets_comprados) as Tickets_comprados,
#     SUM(Tickets_comprados) / SUM(Aforo) as perc_ocupacion_comprados,
#     (SUM(Tickets_comprados) + SUM(Cortesias)) / SUM(Aforo) as perc_ocupacion_emitidos
# 	FROM tmp11
# 	group by Key_evento),
 
#  tmp13 as (
#     SELECT
#     t1.*,
#     t2.Aforo as Aforo_total,
#     t1.Tickets_comprados / t2.Aforo as perc_ocupacion_comprados,
#     (t1.Tickets_comprados + t1.Cortesias) / t2.Aforo as perc_ocupacion_emitidos
#     FROM tmp11 t1
#     LEFT JOIN tmp12 t2
#       ON (t1.Key_evento = t2.Key_evento)

# ''')

# display(tmp)

In [ ]:
# tmp = eventos.filter((col('Key_evento').like('%10229362259188%'))) #&
#                      #(col('Aforo').like('%1392%')))
# # tmp = eventos.select('Key_evento','perc_ocupacion_emitidos').\
# #     orderBy(desc('perc_ocupacion_emitidos')) #filter(col('perc_ocupacion_comprados') > 1)
# display(tmp)

In [ ]:
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Requerimientos/Eventos_montados/"
write_table = eventos.coalesce(1) # coalesce indica que la tabla se va a entregar en una sola particion
write_table.write.mode("overwrite").parquet(inpath)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]
 
# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_Eventos_Montados.parquet")

True

In [ ]:
eventos_movistar = spark.sql('''

WITH tmp11 as (
    SELECT
    Key_evento,
    Key_aforo,
    Nuevo_aforo as Aforo,    
    SUM(NET_SOLD_C_QTY) Cortesias,
    SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados
    FROM sales_vw s
    GROUP BY Key_evento,Key_aforo,Nuevo_aforo),
 
 tmp12 as (
	SELECT 
    Key_evento,
    SUM(Aforo) as Aforo,
    SUM(Tickets_comprados) / SUM(Aforo) as perc_ocupacion_comprados,
    (SUM(Tickets_comprados) + SUM(Cortesias)) / SUM(Aforo) as perc_ocupacion_emitidos
	FROM tmp11
	group by Key_evento),
 
 tmp13 as (
    SELECT
    t1.*,
    t2.Aforo as Aforo_total,
    t1.Tickets_comprados / t1.Aforo as perc_ocupacion_comprados_cl,
    (t1.Tickets_comprados + t1.Cortesias) / t1.Aforo as perc_ocupacion_emitidos_cl,
    t2.perc_ocupacion_comprados,
    t2.perc_ocupacion_emitidos
    FROM tmp11 t1
    LEFT JOIN tmp12 t2
      ON (t1.Key_evento = t2.Key_evento)),
 
 eventos as (
    SELECT
    s.Key_evento,
    s.Key_aforo,
    s.LOGICAL_SEAT_CATEGORY,--
    s.T_PRODUCT_ID AS CODIGO_DEL_EVENTO,
    UPPER(s.PRODUCT) AS Evento,
    s.T_PERFORMANCE_ID as CODIGO_SESION,
    s.ORGANIZATION AS Organismo,
    s.LICENCE_NUMBER AS CODIGO_PULEP,
    s.ACTIVITY as Segmento,--
    s.TOPIC_PPTO as Tema,
    s.SUB_TOPIC as Sub_Tema,
    '' as Tarifa,
    --TO_CHAR(TO_DATE(REPLACE(s.PRODUCT_DATE,'.','/'),'DD/MM/RR'), 'DD/MM/YY') AS FECHA_DEL_EVENTO,
    s.Fecha_evento,
    s.PRODUCT_TIME as Hora_evento,
    s.SITE AS Venue,
    s.TOWN_N as Ciudad,
    s.INVOICE_CONTACT_FMT_NAME as Promotor,
    s.PROMOTION,--
    AVG(s.UNIT_AMT_ITX) as Precio_Full,
    --(SUM(s.NET_SOLD_T_QTY) - SUM(s.NET_SOLD_C_QTY)) * AVG(s.UNIT_AMT_ITX) AS GROSS_POTENCIAL,
    ''  AS GROSS_POTENCIAL,
    SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados,
    SUM(s.NET_SOLD_C_QTY) AS Cortesias,
    SUM(s.NET_SOLD_T_QTY) AS Total_emitido,
    SUM(s.NET_SOLD_TKT_AMT_ITX) AS Recaudo,
    --MIN(s.REFERENCE_DATE) As FECHA_VALIDACION,
    '' AS FECHA_VALIDACION,
    --MAX(s.REFERENCE_DATE) As ULTIMA_FECHA_VALIDACION,
    '' AS ULTIMA_FECHA_VALIDACION,
    SUM(s.CHARGE_AMT_ITX) AS Booking_fee_con_impuesto,
    SUM(s.CHARGE_AMT_ETX) AS Booking_fee_sin_impuesto
    FROM sales_vw s
    WHERE YEAR(Fecha_evento) IN (2023,2024)
      AND SITE = 'Movistar Arena - Bogotá - Dg. 61c #26-36'
      AND s.T_OPERATION_TYPE IN ('SALE','REFUND')
      AND s.T_OPERATION_KIND IN ('SINGLE_ENTRY','PRODUCT_COMPOSITION','SIMPLE_PRODUCT')--'COMPOSED_PRODUCT' 
      AND s.PRODUCT_CODE != 'CAN_INS'
      --AND Key_evento like '%10229406401500%'
    GROUP BY
    s.Key_evento,s.Key_aforo,s.LOGICAL_SEAT_CATEGORY,s.T_PRODUCT_ID,s.PRODUCT,s.T_PERFORMANCE_ID,s.ORGANIZATION,
    s.LICENCE_NUMBER,s.ACTIVITY,s.TOPIC_PPTO,s.SUB_TOPIC,s.Fecha_evento,s.PRODUCT_TIME,s.TOWN_N,s.SITE,s.INVOICE_CONTACT_FMT_NAME,s.PROMOTION
 )
 
SELECT
e.*,
lm.LOCALIDAD_GP,
t.Aforo as Aforo_por_CL,
t.Aforo_total as Aforo_disponible,
t.perc_ocupacion_comprados_cl,
t.perc_ocupacion_emitidos_cl,
t.perc_ocupacion_comprados,
t.perc_ocupacion_emitidos
FROM eventos e
LEFT JOIN tmp13 t
  ON (e.Key_aforo == t.Key_aforo)
LEFT JOIN localidad_movistar_vw lm
  ON (e.LOGICAL_SEAT_CATEGORY == lm.LOGICAL_SEAT_CATEGORY)
ORDER BY Key_aforo

''')


from pyspark.sql.functions import expr

eventos_movistar = eventos_movistar.withColumn(
    "Fecha Actualizacion",
    expr("current_timestamp() - interval 5 hours")
)

eventos_movistar = eventos_movistar \
    .withColumnRenamed('CODIGO_PULEP', 'Código_pulep') \
    .withColumnRenamed('CODIGO_DEL_EVENTO', 'Código_del_evento') \
    .withColumnRenamed('LOGICAL_SEAT_CATEGORY', 'Categoría_Lógica') \
    .withColumnRenamed('Cortesias', 'Cortesías')


#eventos.createOrReplaceTempView("eventos_vw")
print(eventos_movistar.count())
#display(eventos.limit(100))

8103


In [ ]:
tmp = eventos_movistar.filter(col('Key_evento').like('%10229428955305%'))\
#tmp = eventos_movistar.filter(col('Evento').like('%BOOGIE%'))
display(tmp)


Key_evento,Key_aforo,Categoría_Lógica,Código_del_evento,Evento,CODIGO_SESION,Organismo,Código_pulep,Segmento,Tema,Sub_Tema,Tarifa,Fecha_evento,Hora_evento,Venue,Ciudad,Promotor,PROMOTION,Precio_Full,GROSS_POTENCIAL,Tickets_comprados,Cortesías,Total_emitido,Recaudo,FECHA_VALIDACION,ULTIMA_FECHA_VALIDACION,Booking_fee_con_impuesto,Booking_fee_sin_impuesto,LOCALIDAD_GP,Aforo_por_CL,Aforo_disponible,perc_ocupacion_comprados_cl,perc_ocupacion_emitidos_cl,perc_ocupacion_comprados,perc_ocupacion_emitidos,Fecha Actualizacion
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 201 & 219 VISTA PARCIAL,PISO 2 SECCIÓN 201 & 219 VISTA PARCIAL,10229428955305,JUANES BOGOTÁ - JUANES COLOMBIA,10229428959042,Sueño Estéreo,DFJ819,Eventos,Conciertos Arena,ROCK,,2024-05-24T20:00:00Z,20:00,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,PROMOTORA COLOMBIA S.A.S ( 8635388 ),null,206451.21951219512,,165.0,2.0,167.0,3.4485E7,,,6171000.0,5185705.0,PISO 2,192.0,11101.0,0.859375,0.8697916666666666,0.9764886046302135,0.997657868660481,2024-05-30T09:54:13.62Z
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 202-204 & 216-218,PISO 2 SECCIÓN 202-204 & 216-218,10229428955305,JUANES BOGOTÁ - JUANES COLOMBIA,10229428959042,Sueño Estéreo,DFJ819,Eventos,Conciertos Arena,ROCK,,2024-05-24T20:00:00Z,20:00,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,PROMOTORA COLOMBIA S.A.S ( 8635388 ),null,225336.0,,239.0,20.0,259.0,5.4731E7,,,1.01432E7,8523691.0,PISO 2,1551.0,11101.0,0.9871050934880722,1.0,0.9764886046302135,0.997657868660481,2024-05-30T09:54:13.62Z
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 202-204 & 216-218,PISO 2 SECCIÓN 202-204 & 216-218,10229428955305,JUANES BOGOTÁ - JUANES COLOMBIA,10229428959042,Sueño Estéreo,DFJ819,Eventos,Conciertos Arena,ROCK,,2024-05-24T20:00:00Z,20:00,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,PROMOTORA COLOMBIA S.A.S ( 8635388 ),null,229000.0,,546.0,0.0,546.0,1.25034E8,,,2.23314E7,1.87658655E7,PISO 2,1551.0,11101.0,0.9871050934880722,1.0,0.9764886046302135,0.997657868660481,2024-05-30T09:54:13.62Z
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 202-204 & 216-218,PISO 2 SECCIÓN 202-204 & 216-218,10229428955305,JUANES BOGOTÁ - JUANES COLOMBIA,10229428959042,Sueño Estéreo,DFJ819,Eventos,Conciertos Arena,ROCK,,2024-05-24T20:00:00Z,20:00,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,PROMOTORA COLOMBIA S.A.S ( 8635388 ),Movistar Preferencial - Juanes Bogotá,229000.0,,248.0,0.0,248.0,5.6792E7,,,1.01432E7,8523691.0,PISO 2,1551.0,11101.0,0.9871050934880722,1.0,0.9764886046302135,0.997657868660481,2024-05-30T09:54:13.62Z
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 202-204 & 216-218,PISO 2 SECCIÓN 202-204 & 216-218,10229428955305,JUANES BOGOTÁ - JUANES COLOMBIA,10229428959042,Sueño Estéreo,DFJ819,Eventos,Conciertos Arena,ROCK,,2024-05-24T20:00:00Z,20:00,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,PROMOTORA COLOMBIA S.A.S ( 8635388 ),Preventa Aval - Juanes,229000.0,,498.0,0.0,498.0,1.14042E8,,,2.03682E7,1.7116122E7,PISO 2,1551.0,11101.0,0.9871050934880722,1.0,0.9764886046302135,0.997657868660481,2024-05-30T09:54:13.62Z
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 205-206 & 214-215,PISO 2 SECCIÓN 205-206 & 214-215,10229428955305,JUANES BOGOTÁ - JUANES COLOMBIA,10229428959042,Sueño Estéreo,DFJ819,Eventos,Conciertos Arena,ROCK,,2024-05-24T20:00:00Z,20:00,Movistar Arena - Bogotá - Dg. 61c #26-36,BOGOTA,PROMOTORA COLOMBIA S.A.S ( 8635388 ),Preventa Aval - Juanes,199000.0,,279.0,0.0,279.0,5.5521E7,,,9932400.0,8346564.0,PISO 2,989.0,11101.0,0.9696663296258847,1.0,0.9764886046302135,0.997657868660481,2024-05-30T09:54:13.62Z
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 205-206 & 214-215,PISO 2 SECCIÓN 205-206 & 214-215,10229428955305,JUANES BOGOTÁ - JUANES COLOMBIA,10229428959042,Sueño Estéreo,DFJ819,Eventos,Conciertos Arena,ROCK,,2024-05-24T20:0

'PISO 3 SECCIÓN 307-313'

In [ ]:
# tmp = spark.sql('''
#     SELECT
#     Key_evento,
#     Key_aforo,
#     Nuevo_aforo as Aforo,    
#     SUM(NET_SOLD_C_QTY) Cortesias,
#     SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados
#     FROM sales_vw s
#     WHERE Key_evento LIKE '%10229428955305%'
#     GROUP BY Key_evento,Key_aforo,Nuevo_aforo
# ''')

# display(tmp)
# '10229119820162|10229119824808|FLOOR 3 SECTION 308 - 312'
# ''

Key_evento,Key_aforo,Aforo,Cortesias,Tickets_comprados
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 202-204 & 216-218,1551.0,20.0,1531.0
10229428955305|10229428959042,10229428955305|10229428959042|PLATEA 104-106 FULL,748.0,60.0,688.0
10229428955305|10229428959042,10229428955305|10229428959042|PISO 3 SECCIÓN 303-304 & 316-317,1392.0,0.0,1392.0
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 205-206 & 214-215,989.0,30.0,959.0
10229428955305|10229428959042,10229428955305|10229428959042|TRIBUNA FAN NORTE,70.0,0.0,70.0
10229428955305|10229428959042,10229428955305|10229428959042|PISO 3 SECCIÓN 302-318,602.0,0.0,602.0
10229428955305|10229428959042,10229428955305|10229428959042|PISO 3 SECCIÓN 307-313,1230.0,0.0,1230.0
10229428955305|10229428959042,10229428955305|10229428959042|PISO 2 SECCIÓN 207-213,1107.0,0.0,1107.0
10229428955305|10229428959042,10229428955305|10229428959042|PLATEA 102 (6 PRIMERAS FILAS),120.0,10.0,110.0
10229428955305|10229428959042,10229428955305|10229428959042|PISO 3 SECCIÓN 305-306 & 314-315,1234.0,0.0,1234.0


''

In [ ]:
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/temp/Movistar/"
write_table = eventos_movistar.coalesce(1) # coalesce indica que la tabla se va a entregar en una sola particion
write_table.write.mode("overwrite").parquet(inpath)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]
 
# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Movistar_Arena")

True